In [111]:
from pymongo import MongoClient
import os
import json

In [112]:
def connect_mongodb(uri,db_name, collection_name):
    # Connect to MongoDB 
    client = MongoClient(uri)
    
    # Select the database and collection
    db = client[db_name]
    collection = db[collection_name]
    return collection
def get_files(path: str):
    """
    Get all file names under the specified path.
    
    :param path: The root directory path to search.
    :return: A list of file names.
    """
    try:
        return [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    except FileNotFoundError:
        print(f"Error: The path '{path}' does not exist.")
        return []
    except PermissionError:
        print(f"Error: Permission denied for path '{path}'.")
        return []
def update_sensitive_data_in_mongodb(collection, json_data, document_id):
    """
    Updates MongoDB document with 'sensitive_data_detected' field.

    Steps:
    1. Extracts the value under "sensitive_data_detected".
    2. Ensures the extracted value is correctly formatted as either:
       - A list of dictionaries (Case-1)
       - A string message (Case-2)
    3. Updates the MongoDB document at _id=document_id.

    :param collection: MongoDB collection object.
    :param json_data: The JSON data containing 'sensitive_data_detected'.
    :param document_id: The _id of the document to update.
    """
    # Step 1: Extract "sensitive_data_detected" value
    sensitive_data = json_data.get("sensitive_data_detected", "no sensitive data detected")

    # Step 2: Ensure consistent format
    if isinstance(sensitive_data, list):  
        # Case-1: List of dictionaries (valid sensitive data list)
        update_value = {"sensitive_data_detected": sensitive_data}
    elif isinstance(sensitive_data, str):  
        # Case-2: String message ("no sensitive data detected")
        update_value = {"sensitive_data_detected": {"message": sensitive_data}}
    else:
        print(f"⚠ Unexpected format for 'sensitive_data_detected' in document {document_id}. Skipping update.")
        return

    # Step 3: Update MongoDB document
    update_query = {"_id": document_id}
    update_data = {"$set": update_value}

    result = collection.update_one(update_query, update_data)

    # Step 4: Print result message
    if result.matched_count > 0:
        print(f"✅ Successfully updated document with _id: {document_id}")
    else:
        print(f"⚠ Document with _id: {document_id} not found.")
def read_json_from_txt(file_path):
    """
    Reads a .txt file containing a JSON structure and parses it into a dictionary.

    :param file_path: Path to the .txt file.
    :return: Parsed JSON dictionary if successful, else an empty dictionary {}.
    """
    try:
        # Read file content and handle encoding issues
        with open(file_path, "r", encoding="utf-8-sig") as file:  
            content = file.read().strip()  

        # Check if file is empty
        if not content:
            print(f"Error: File {file_path} is empty.")
            return {}

        # Remove possible markdown code block markers (```json ... ```)
        if content.startswith("```json"):
            content = content[len("```json"):].strip("`").strip()
        elif content.startswith("```"):
            content = content[len("```"):].strip("`").strip()

        # Ensure the content is a JSON object
        if not content.startswith("{") or not content.endswith("}"):
            print(f"Error: File {file_path} does not contain a valid JSON structure.")
            return {}

        # Convert the cleaned content into JSON
        json_data = json.loads(content)
        return json_data

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file {file_path}: {e}")
        return {}
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return {}
    except Exception as e:
        print(f"Unexpected error: {e}")
        return {}

In [113]:
mongoDB_uri = 'mongodb://192.168.1.14:27017'
mongoDB_database = 'wearable-project' 
mongoDB_collection = 'wearable-app'

In [114]:
# Connect to the MongoDB collection
collection = connect_mongodb(mongoDB_uri,mongoDB_database,mongoDB_collection)

In [115]:
file_path = r"C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json"

In [116]:
file_name_arr = get_files(file_path)
print(len(file_name_arr))
# print(file_name_arr)

641


In [117]:
for i in range(len(file_name_arr)):
    print("---------------------------- Loop-"+str(i)+" ----------------------------")
    file_name = file_name_arr[i]
    print("File name: ",file_name)
    package_name =  file_name.rsplit(".", 1)[0]
    print("Package name: ",package_name)
    sensitive_data_file_path = file_path+"\\"+file_name
    print("Sensitive data file path: ",sensitive_data_file_path)
    json_data = read_json_from_txt(sensitive_data_file_path)
#     print(json_data)
    update_sensitive_data_in_mongodb(collection, json_data, package_name)
#     break

---------------------------- Loop-0 ----------------------------
File name:  adarshurs.android.vlcmobileremote.txt
Package name:  adarshurs.android.vlcmobileremote
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\adarshurs.android.vlcmobileremote.txt
✅ Successfully updated document with _id: adarshurs.android.vlcmobileremote
---------------------------- Loop-1 ----------------------------
File name:  app.EnForma.fitcoapp.net.txt
Package name:  app.EnForma.fitcoapp.net
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\app.EnForma.fitcoapp.net.txt
✅ Successfully updated document with _id: app.EnForma.fitcoapp.net
---------------------------- Loop-2 ----------------------------
File name:  app.entrenamientogenetico.android.fitcoapp.net.txt
Package name:  app.entrenamientogenetico.android.fitcoapp.net
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privac

✅ Successfully updated document with _id: com.basemsalloum.goodfitness.goodfitness
---------------------------- Loop-63 ----------------------------
File name:  com.basicfit.trainingApp.txt
Package name:  com.basicfit.trainingApp
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.basicfit.trainingApp.txt
✅ Successfully updated document with _id: com.basicfit.trainingApp
---------------------------- Loop-64 ----------------------------
File name:  com.bbfit.txt
Package name:  com.bbfit
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.bbfit.txt
✅ Successfully updated document with _id: com.bbfit
---------------------------- Loop-65 ----------------------------
File name:  com.beatxp.dafit.txt
Package name:  com.beatxp.dafit
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.beatxp.dafit.txt
✅ S

✅ Successfully updated document with _id: com.evlofitness
---------------------------- Loop-129 ----------------------------
File name:  com.evolutionwellness.app.celebrityfitness.txt
Package name:  com.evolutionwellness.app.celebrityfitness
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.evolutionwellness.app.celebrityfitness.txt
✅ Successfully updated document with _id: com.evolutionwellness.app.celebrityfitness
---------------------------- Loop-130 ----------------------------
File name:  com.evolutionwellness.app.fitnessfirst.txt
Package name:  com.evolutionwellness.app.fitnessfirst
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.evolutionwellness.app.fitnessfirst.txt
✅ Successfully updated document with _id: com.evolutionwellness.app.fitnessfirst
---------------------------- Loop-131 ----------------------------
File name:  com.eztech.fitness.txt


✅ Successfully updated document with _id: com.fitworkout.lossweight.homeworkout.fitnessapp
---------------------------- Loop-191 ----------------------------
File name:  com.fitze.txt
Package name:  com.fitze
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.fitze.txt
✅ Successfully updated document with _id: com.fitze
---------------------------- Loop-192 ----------------------------
File name:  com.fitzeee.fitzeeerun.txt
Package name:  com.fitzeee.fitzeeerun
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.fitzeee.fitzeeerun.txt
✅ Successfully updated document with _id: com.fitzeee.fitzeeerun
---------------------------- Loop-193 ----------------------------
File name:  com.fitzeee.menworkout.txt
Package name:  com.fitzeee.menworkout
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.fitze

✅ Successfully updated document with _id: com.jp.fitness.biceps.exrcise
---------------------------- Loop-255 ----------------------------
File name:  com.jquiz.english.txt
Package name:  com.jquiz.english
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.jquiz.english.txt
✅ Successfully updated document with _id: com.jquiz.english
---------------------------- Loop-256 ----------------------------
File name:  com.justfit.ymove.affitlife.txt
Package name:  com.justfit.ymove.affitlife
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.justfit.ymove.affitlife.txt
✅ Successfully updated document with _id: com.justfit.ymove.affitlife
---------------------------- Loop-257 ----------------------------
File name:  com.jyfitness.iron_mastery.txt
Package name:  com.jyfitness.iron_mastery
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\s

✅ Successfully updated document with _id: com.myfitnesscoach
---------------------------- Loop-311 ----------------------------
File name:  com.myfitnesspal.android.txt
Package name:  com.myfitnesspal.android
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.myfitnesspal.android.txt
✅ Successfully updated document with _id: com.myfitnesspal.android
---------------------------- Loop-312 ----------------------------
File name:  com.myhexfit.app.txt
Package name:  com.myhexfit.app
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.myhexfit.app.txt
✅ Successfully updated document with _id: com.myhexfit.app
---------------------------- Loop-313 ----------------------------
File name:  com.mypthub.colourfit.txt
Package name:  com.mypthub.colourfit
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.m

✅ Successfully updated document with _id: com.pushpress.punchkingfitnesssunset
---------------------------- Loop-376 ----------------------------
File name:  com.qingniu.feelfit.txt
Package name:  com.qingniu.feelfit
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.qingniu.feelfit.txt
✅ Successfully updated document with _id: com.qingniu.feelfit
---------------------------- Loop-377 ----------------------------
File name:  com.qingniu.fitindex.txt
Package name:  com.qingniu.fitindex
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.qingniu.fitindex.txt
✅ Successfully updated document with _id: com.qingniu.fitindex
---------------------------- Loop-378 ----------------------------
File name:  com.qingniu.inevifit.txt
Package name:  com.qingniu.inevifit
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_ap

✅ Successfully updated document with _id: com.transformfitness
---------------------------- Loop-443 ----------------------------
File name:  com.treshna.memberportal.fusionfitness.txt
Package name:  com.treshna.memberportal.fusionfitness
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.treshna.memberportal.fusionfitness.txt
✅ Successfully updated document with _id: com.treshna.memberportal.fusionfitness
---------------------------- Loop-444 ----------------------------
File name:  com.ubivelox.uvophone.txt
Package name:  com.ubivelox.uvophone
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\com.ubivelox.uvophone.txt
✅ Successfully updated document with _id: com.ubivelox.uvophone
---------------------------- Loop-445 ----------------------------
File name:  com.unated.txt
Package name:  com.unated
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-p

✅ Successfully updated document with _id: digifit.android.virtuagym.pro.fitnessparksuperclub
---------------------------- Loop-512 ----------------------------
File name:  digifit.android.virtuagym.pro.greaterjolietymcasuperclub.txt
Package name:  digifit.android.virtuagym.pro.greaterjolietymcasuperclub
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\digifit.android.virtuagym.pro.greaterjolietymcasuperclub.txt
✅ Successfully updated document with _id: digifit.android.virtuagym.pro.greaterjolietymcasuperclub
---------------------------- Loop-513 ----------------------------
File name:  digifit.android.virtuagym.pro.gymnasium.txt
Package name:  digifit.android.virtuagym.pro.gymnasium
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\digifit.android.virtuagym.pro.gymnasium.txt
✅ Successfully updated document with _id: digifit.android.virtuagym.pro.gymnasium
-------

✅ Successfully updated document with _id: kr.co.openit.openrider
---------------------------- Loop-579 ----------------------------
File name:  kr.fitt.fmes.txt
Package name:  kr.fitt.fmes
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\kr.fitt.fmes.txt
✅ Successfully updated document with _id: kr.fitt.fmes
---------------------------- Loop-580 ----------------------------
File name:  kz.maestrosultan.fitjournal.txt
Package name:  kz.maestrosultan.fitjournal
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensitive_data_wearable_app_json\kz.maestrosultan.fitjournal.txt
✅ Successfully updated document with _id: kz.maestrosultan.fitjournal
---------------------------- Loop-581 ----------------------------
File name:  loseweight.weightloss.workout.fitness.txt
Package name:  loseweight.weightloss.workout.fitness
Sensitive data file path:  C:\Users\ASUS\anaconda3\wearable-privacy-compliance\sensit